# **Path Optimization using AI**

**Team Members**

1. Edwin Johny Paul
2. Sethulakshmi Kochuchirayil Babu
3. Sandeep Pidugu
4. Parth Pareshbhai Vekariya

**Accessing Dataset**

OSMnx library is used to import the required data with which the shortest path is found between two nodes/places.

In [ ]:
!pip install requests folium geopy
!pip install pyproj

!pip install requests osmnx folium geopy networkx
!pip install networkx osmnx requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.4/99.4 kB 3.8 MB/s eta 0:00:00


**Importing necessary libraries**

- **requests**: For making API calls (e.g., GraphHopper, OpenWeather).
- **folium**: For creating interactive maps.
- **heapq**: For priority queue in A* algorithm.
- **geopy.Nominatim**: To geocode place names to coordinates.
- **IPython.display**: To render maps in Jupyter Notebook.
- **time**: For adding delays, e.g., during retries.

In [ ]:
import requests
import folium
import heapq
from geopy.geocoders import Nominatim
from IPython.display import display
import time

In [ ]:
# API Keys
GRAPH_HOPPER_API_KEY = "YOUR API KEY"
OPENWEATHER_API_KEY = "YOUR API KEY"

**A* for path finding**

In [ ]:
def astar(start, goal, graph, heuristic):
    open_list = []
    closed_list = set()
    came_from = {}
    g_score = {start: 0}
    f_score = {start: heuristic(start, goal)}
    heapq.heappush(open_list, (f_score[start], start))

    while open_list:
        _, current = heapq.heappop(open_list)

        if current == goal:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.append(start)
            path.reverse()
            return path

        closed_list.add(current)

        for neighbor, cost in graph.get(current, {}).items():
            if neighbor in closed_list:
                continue
            tentative_g_score = g_score[current] + cost
            if neighbor not in g_score or tentative_g_score < g_score[neighbor]:
                came_from[neighbor] = current
                g_score[neighbor] = tentative_g_score
                f_score[neighbor] = tentative_g_score + heuristic(neighbor, goal)
                heapq.heappush(open_list, (f_score[neighbor], neighbor))

    return None

**Applying Euclidean distance heuristic**

In [ ]:
def euclidean_heuristic(node, goal):
    x1, y1 = node
    x2, y2 = goal
    return ((x2 - x1) ** 2 + (y2 - y1) ** 2) ** 0.5

**Adding Real Time Traffic Informations using GraphHopper API**

In [ ]:
def get_graphhopper_route(start_coords, end_coords, alternative=True):
    start_lat, start_lon = start_coords
    end_lat, end_lon = end_coords
    url = f"https://graphhopper.com/api/1/route?point={start_lat},{start_lon}&point={end_lat},{end_lon}&type=json&vehicle=car&locale=en&key={GRAPH_HOPPER_API_KEY}&alternatives={str(alternative).lower()}"
    response = requests.get(url)
    try:
        data = response.json()
        if "paths" in data:
            return data["paths"]
        else:
            print(f"GraphHopper API error: {data.get('message', 'Unexpected response')}")
            return None
    except Exception as e:
        print(f"Failed to fetch route data: {e}")
        return None

**Adding Weather conditions using OpenWeatherMap API**

In [ ]:
def get_weather(location_name):
    url = f"https://api.openweathermap.org/data/2.5/weather?q={location_name}&appid={OPENWEATHER_API_KEY}&units=metric"
    response = requests.get(url)
    try:
        data = response.json()
        if data.get("cod") == 200:
            return {
                "description": data["weather"][0]["description"],
                "temp": data["main"]["temp"],
                "wind_speed": data["wind"]["speed"]
            }
        else:
            print(f"Weather API error: {data.get('message', 'Unexpected response')}")
            return None
    except Exception as e:
        print(f"Failed to fetch weather data: {e}")
        return None

def display_weather(start_weather, end_weather):
    print("Weather Information:")
    print(f"\nStart Location Weather:")
    print(f"Condition: {start_weather['description'].capitalize()}")
    print(f"Temperature: {start_weather['temp']}°C")
    print(f"Wind Speed: {start_weather['wind_speed']} m/s")

    print(f"\nEnd Location Weather:")
    print(f"Condition: {end_weather['description'].capitalize()}")
    print(f"Temperature: {end_weather['temp']}°C")
    print(f"Wind Speed: {end_weather['wind_speed']} m/s")

**Estimating Traffic Flow**

In [ ]:
def estimate_traffic_flow(route_time, expected_time):
    delay = (route_time - expected_time) / 60
    if delay <= 0:
        return "green"  # Low traffic
    elif delay <= 10:
        return "yellow"  # Moderate traffic
    else:
        return "red"  # Heavy traffic

**Visualising Routes**

In [ ]:
def visualize_route(route_data, start_coords, end_coords):
    route_map = folium.Map(location=start_coords, zoom_start=14)

    folium.Marker(location=start_coords, popup="Start", icon=folium.Icon(color="green")).add_to(route_map)
    folium.Marker(location=end_coords, popup="End", icon=folium.Icon(color="red")).add_to(route_map)

    if route_data:
        for idx, path in enumerate(route_data):
            polyline = path.get("points", None)
            if polyline:
                decoded_points = decode_polyline(polyline)
                if decoded_points:
                    route_distance = path.get("distance", 0) / 1000
                    expected_time = route_distance / 30 * 60
                    route_time = path.get("time", 0) / 1000

                    traffic_flow = estimate_traffic_flow(route_time, expected_time)

                    folium.PolyLine(
                        decoded_points,
                        color=traffic_flow,
                        weight=5,
                        popup=f"Route {idx+1}: {route_distance:.2f} km, {route_time/60:.1f} mins"
                    ).add_to(route_map)
                else:
                    print(f"Failed to decode polyline for Route {idx+1}")
            else:
                print(f"No polyline data for Route {idx+1}")
    else:
        print("No route data available.")

    # Adding traffic flow legend
    legend_html = """
    <div style="position: fixed; bottom: 10px; left: 10px; width: 200px; height: 120px; background-color: white;
    border: 2px solid black; z-index:9999; font-size: 14px; padding: 10px;">
        <b>Traffic Flow Legend</b><br>
        <i style="background-color: green; width: 20px; height: 20px; display: inline-block;"></i> Low Traffic<br>
        <i style="background-color: yellow; width: 20px; height: 20px; display: inline-block;"></i> Moderate Traffic<br>
        <i style="background-color: red; width: 20px; height: 20px; display: inline-block;"></i> Heavy Traffic
    </div>
    """
    route_map.get_root().html.add_child(folium.Element(legend_html))

    display(route_map)


**Geocoding the location using OSM Nominatim tool and fetching the lattittude and longitude using Polyline**

In [ ]:
# Decode GraphHopper polyline
def decode_polyline(encoded):
    coords = []
    index, lat, lng = 0, 0, 0
    while index < len(encoded):
        shift, result = 0, 0
        while True:
            b = ord(encoded[index]) - 63
            index += 1
            result |= (b & 0x1F) << shift
            shift += 5
            if b < 0x20:
                break
        lat += ~(result >> 1) if result & 1 else result >> 1

        shift, result = 0, 0
        while True:
            b = ord(encoded[index]) - 63
            index += 1
            result |= (b & 0x1F) << shift
            shift += 5
            if b < 0x20:
                break
        lng += ~(result >> 1) if result & 1 else result >> 1

        coords.append((lat / 1e5, lng / 1e5))
    return coords

def geocode_location(location_name):
    geolocator = Nominatim(user_agent="route_finder")
    retries = 3
    for _ in range(retries):
        try:
            location = geolocator.geocode(location_name, timeout=5)
            if location:
                return location.latitude, location.longitude
        except Exception as e:
            print(f"Error geocoding {location_name}: {e}")
            time.sleep(2)
    return None

In [ ]:
if __name__ == "__main__":
    start_location = input("Enter the starting place name: ")
    end_location = input("Enter the ending place name: ")

    start_coords = geocode_location(start_location)
    end_coords = geocode_location(end_location)

    if start_coords and end_coords:
        start_weather = get_weather(start_location)
        end_weather = get_weather(end_location)

        if start_weather and end_weather:
            display_weather(start_weather, end_weather)

        route_data = get_graphhopper_route(start_coords, end_coords, alternative=True)
        if route_data:
            # distance and time for primary route
            total_distance_in_km = route_data[0]['distance'] / 1000
            route_time_in_minutes = route_data[0]['time'] / 1000 / 60

            print(f"Total Distance: {total_distance_in_km:.2f} km")
            print(f"Route Time: {route_time_in_minutes:.2f} minutes")
            print("Route Instructions:")

            cumulative_distance = 0
            for step in route_data[0].get("instructions", []):
                step_distance_km = step['distance'] / 1000
                cumulative_distance += step['distance']
                print(f" - {step['text']} ({step['distance']} meters / {step_distance_km:.2f} km)")

            print(f"Cumulative Distance: {cumulative_distance / 1000:.2f} km")

            visualize_route(route_data, start_coords, end_coords)
        else:
            print("Failed to fetch route data.")
    else:
        print("Failed to geocode one or both locations.")


Enter the starting place name: chatelet
Enter the ending place name: bercy
Weather Information:

Start Location Weather:
Condition: Mist
Temperature: 2.28°C
Wind Speed: 4.63 m/s

End Location Weather:
Condition: Overcast clouds
Temperature: 5.08°C
Wind Speed: 4.12 m/s
Total Distance: 5.49 km
Route Time: 26.01 minutes
Route Instructions:
 - Continue (539.762 meters / 0.54 km)
 - Continue onto Souterrain Grande Boucle (Voie BCL) (299.604 meters / 0.30 km)
 - Keep left onto Souterrain Sortie Hôtel de Ville (Voie S2) (699.046 meters / 0.70 km)
 - Turn right onto Rue de Rivoli (168.269 meters / 0.17 km)
 - Turn left onto Rue Saint-Martin (70.607 meters / 0.07 km)
 - Turn left onto Quai de Gesvres (2662.582 meters / 2.66 km)
 - Keep right onto Quai de la Rapée (262.747 meters / 0.26 km)
 - Continue onto Quai de Bercy (22.107 meters / 0.02 km)
 - Turn left onto Boulevard de Bercy (552.138 meters / 0.55 km)
 - Turn right (128.959 meters / 0.13 km)
 - Turn sharp left (88.253 meters / 0.09 km)
 

# **ALL POSSIBLE ROUTES**

In [ ]:
!pip install osmnx networkx geopy matplotlib

In [ ]:
!pip install folium osmnx geopy

In [ ]:
import folium
import osmnx as ox
from geopy.distance import great_circle
from heapq import heappop, heappush
from geopy.geocoders import Nominatim

**Geocoding the location by finding coordinates of the place using Nominatim**

In [ ]:
def geocode_location(location_name):
    geolocator = Nominatim(user_agent="route_finder")
    try:
        location = geolocator.geocode(location_name, timeout=5)
        if location:
            return location.latitude, location.longitude
        else:
            print(f"Geocoding failed for: {location_name}")
            return None
    except Exception as e:
        print(f"Error geocoding {location_name}: {e}")
        return None

**Fetching street network graph**

In [ ]:
def get_graph_for_location(location_name, radius=1000):
    location_coords = geocode_location(location_name)
    if location_coords:
        graph = ox.graph_from_point(location_coords, dist=radius, network_type='all')
        return graph
    return None

**Applying A* star algorithm to find all the paths avoiding repetitive and shortest paths**

In [ ]:
def a_star(graph, start_node, end_node, heuristic, max_paths=5):
    open_list = []
    came_from = {}
    g_score = {node: float('inf') for node in graph.nodes()}
    g_score[start_node] = 0
    f_score = {node: float('inf') for node in graph.nodes()}
    f_score[start_node] = heuristic(graph, start_node, end_node)
    heappush(open_list, (f_score[start_node], start_node))

    found_paths = []
    explored_nodes = set()

    while open_list and len(found_paths) < max_paths:
        _, current_node = heappop(open_list)

        if current_node == end_node:
            path = []
            while current_node in came_from:
                path.append(current_node)
                current_node = came_from[current_node]
            path.append(start_node)
            found_paths.append(path[::-1])

            explored_nodes.update(path)

        # Exploring all neighbors of the current node
        for neighbor in graph.neighbors(current_node):
            penalty = 10 if neighbor in explored_nodes else 0
            tentative_g_score = g_score[current_node] + graph[current_node][neighbor].get('length', 1) + penalty

            if tentative_g_score < g_score[neighbor]:
                came_from[neighbor] = current_node
                g_score[neighbor] = tentative_g_score
                f_score[neighbor] = tentative_g_score + heuristic(graph, neighbor, end_node)
                heappush(open_list, (f_score[neighbor], neighbor))

    return found_paths

**Applying Heuristic on A***

In [ ]:
def heuristic(graph, node, end_node):
    x1, y1 = graph.nodes[node]['x'], graph.nodes[node]['y']
    x2, y2 = graph.nodes[end_node]['x'], graph.nodes[end_node]['y']
    return great_circle((y1, x1), (y2, x2)).meters

**Visualisation on Map**

In [ ]:
def visualize_routes_on_map(route_data, start_coords, end_coords, graph):
    print(f"Start coordinates: {start_coords}")
    print(f"End coordinates: {end_coords}")

    route_map = folium.Map(location=start_coords, zoom_start=14)

    folium.Marker(location=[start_coords[0], start_coords[1]], popup="Start", icon=folium.Icon(color="green")).add_to(route_map)
    folium.Marker(location=[end_coords[0], end_coords[1]], popup="End", icon=folium.Icon(color="red")).add_to(route_map)

    route_colors = ['blue', 'green', 'orange', 'purple', 'red']

    # Plotting all routes on the map with labels for easy identification of user
    for idx, path in enumerate(route_data):
        route_coords = [(graph.nodes[node]['y'], graph.nodes[node]['x']) for node in path]

        total_distance = 0
        for i in range(len(path) - 1):
            start_lat, start_lon = graph.nodes[path[i]]['y'], graph.nodes[path[i]]['x']
            end_lat, end_lon = graph.nodes[path[i + 1]]['y'], graph.nodes[path[i + 1]]['x']
            total_distance += great_circle((start_lat, start_lon), (end_lat, end_lon)).meters

        route_distance_km = total_distance / 1000
        expected_time = route_distance_km / 30 * 60
        popup_text = f"Route {idx+1}: {route_distance_km:.2f} km, {expected_time:.1f} mins"

        # Adding polyline to the map(with colour to identify diff routes)
        folium.PolyLine(route_coords,
                        color=route_colors[idx % len(route_colors)],
                        weight=5,
                        popup=popup_text).add_to(route_map)

        mid_point = route_coords[len(route_coords) // 2]
        folium.Marker(location=mid_point, popup=f"Route {idx+1}", icon=folium.Icon(color="blue", icon="info-sign")).add_to(route_map)

    return route_map

**Displaying the results**

In [ ]:
def display_routes(graph, all_routes, start_location, end_location, start_coords, end_coords):
    if not all_routes:
        print("No routes found.")
        return

    route_map = visualize_routes_on_map(all_routes, start_coords, end_coords, graph)

    display(route_map)

    print(f"Total routes found: {len(all_routes)}")
    for i, path in enumerate(all_routes):
        route_coords = [(graph.nodes[node]['y'], graph.nodes[node]['x']) for node in path]
        total_distance = 0
        for i in range(len(path) - 1):
            start_lat, start_lon = graph.nodes[path[i]]['y'], graph.nodes[path[i]]['x']
            end_lat, end_lon = graph.nodes[path[i + 1]]['y'], graph.nodes[path[i + 1]]['x']
            total_distance += great_circle((start_lat, start_lon), (end_lat, end_lon)).meters
        route_distance_km = total_distance / 1000  # km
        expected_time = route_distance_km / 30 * 60  # in minutes

        print(f"Route {i+1}:")
        print(f"  Distance: {route_distance_km:.2f} km")
        print(f"  Estimated Time: {expected_time:.1f} mins")
        print("-" * 40)

**Main function**

In [ ]:
if __name__ == "__main__":

    graph_start = get_graph_for_location(start_location, radius=3000)
    if not graph_start:
        print(f"Could not load the graph for {start_location}.")
    else:
        start_coords = geocode_location(start_location)
        if not start_coords:
            print("Error geocoding start location.")
            exit()
        start_node = ox.distance.nearest_nodes(graph_start, start_coords[1], start_coords[0])

        graph_end = get_graph_for_location(end_location, radius=3000)
        if not graph_end:
            print(f"Could not load the graph for {end_location}.")
        else:
            end_coords = geocode_location(end_location)
            if not end_coords:
                print("Error geocoding end location.")
                exit()
            end_node = ox.distance.nearest_nodes(graph_start, end_coords[1], end_coords[0])

            all_routes = a_star(graph_start, start_node, end_node, heuristic, max_paths=5)

            display_routes(graph_start, all_routes, start_location, end_location, start_coords, end_coords)

Start coordinates: (11.8763836, 75.3737973)
End coordinates: (9.6287383, 76.64553257390992)


Total routes found: 1
Route 86:
  Distance: 7.69 km
  Estimated Time: 15.4 mins
----------------------------------------


In [ ]:
# API Keys
GRAPH_HOPPER_API_KEY = "8e462f8a-7046-4bc3-9dbc-1ae25e990383"
OPENWEATHER_API_KEY = "50683b4c3bc21aef7c2436461ee7af10"

We tried to implement Graphhopper API,to involve traffic data and finding the  the alternate paths to get accurate results, but since we are using the free version of the Graphhopper API it is limited to implementing in one route.So we used A* for this by finding and avoiding the shortest path generated and then displaying all the other possible paths and therefore there are slight changes in distance and time for alternate paths when compared to the real world.